In [1]:
# import modules
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import pandas as pd
import ethnicolr as ec
from pathlib import Path
import re

# test queries
test = pd.concat(map(pd.read_csv, [str(Path().resolve().parents[2]) + "\Reference\Indian-Female-Names.csv",
                                   str(Path().resolve().parents[2]) + "\Reference\Indian-Male-Names.csv",
                                   str(Path().resolve().parents[2]) + "\Reference\Indian-Surnames.csv"]))
test = test.dropna()
test['first name'] = test['first name'].map(lambda x: x if x == " " else x.split()[0])
test.reset_index(drop=True)
test = test.drop(test[test['first name'].map(lambda x: any(check in x for check in [",", ".", "@", "-", "`", "0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]))].index)
test.reset_index(drop=True)
test = test.drop(test[test['first name'].map(lambda x: False if x == " " else not bool(re.match('[a-zA-Z]', x)))].index)
test = test.drop_duplicates().reset_index(drop=True)

# dictionary to hold data
data = {
    'First Name' : [],
    'Last Name' : [],
    'Email' : []
}

# open browser and go to directory
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-extensions')
driver = str(Path().resolve().parents[2]) + "\Tools\chromedriver.exe"
timeout = 60
browser = webdriver.Chrome(executable_path=driver, options=chrome_options)

# iterate through all queries
for i in test.index:
    # go to directory
    browser.get('http://contacts.ucalgary.ca/search/advanced?keywords=')
    
    # make next query
    f = test.iat[i, 0]
    l = test.iat[i, 1]
    q = f + " " + l
    
    print("QUERY: %s" %q)

    # find input field and submit query    
    while True:
        try:
            input_field = browser.find_element_by_id('edit-criteria-0-keywords')
            input_field.clear()
            input_field.send_keys(q + Keys.ENTER)
            break
        except:
            browser.quit()
            browser = webdriver.Chrome(executable_path=driver, options=chrome_options)
            browser.get('http://contacts.ucalgary.ca/search/advanced?keywords=')
            print("\tCAN'T FIND INPUT FIELD")
    
    # if results are found
    while True:
        try:
            if "No results found." not in browser.find_element_by_css_selector("div[id^='uofc-module-page'").get_attribute('innerHTML'):
                try:
                    results = browser.find_element_by_id('uofc-table-2').find_element_by_tag_name('tbody').find_elements_by_css_selector("tr[class$='primary-row'")
                except:
                    print("\tNo Results Found. (can't find results datatable)")
                    break
                
                if results == []:
                    print("\tNo Results Found. ('no results' not shown on page)")
                    break
     
                # iterate through each person                    
                for prof in results:
                    # obtain contact info and clean it
                    try:
                        name_col = prof.find_element_by_class_name('uofc-directory-name-cell')
                        email_col = prof.find_element_by_class_name('uofc-directory-email-cell')
                    except:
                        print("\tNo Results Found. (business information found)")
                        break
                    full_name = name_col.find_element_by_tag_name('a').text.split(', ')
                    fname = full_name[1]
                    lname = full_name[0]
                    email = re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+", email_col.get_attribute('innerHTML'), re.I)
                    if email != []:
                        email = email[0]
                    else:
                        email = "No Email"

                    # store contact info in data dictionary
                    data['First Name'] += [fname]
                    data['Last Name'] += [lname]
                    data['Email'] += [email]

                    # print contact info
                    print("\t"+fname, lname, email)
                    
            else:
                print("\tNo Results Found. ('no results' shown on page)")
            break
        except:
            while True:
                try:
                    input_field = browser.find_element_by_id('edit-criteria-0-keywords')
                    input_field.clear()
                    input_field.send_keys(q + Keys.ENTER)
                    break
                except:
                    browser.quit()
                    browser = webdriver.Chrome(executable_path=driver, options=chrome_options)
                    browser.get('http://contacts.ucalgary.ca/search/advanced?keywords=')
                    print("\tCAN'T FIND RESULTS WEB CONTAINER")
                
# close browser
browser.quit()

# create pandas dataframe and remove any duplicate entries, add manually retrieved contact info
df = pd.DataFrame(data)
df = df.drop_duplicates().reset_index(drop=True)
    
# determine ethnicity
df = ec.pred_wiki_name(df,list(df)[1],list(df)[0])
    
# print dataframe and write data to csv file
display(df)
df.to_csv("ucalgary parsed.csv")

Using TensorFlow backend.


QUERY: shivani  
	Shivani Narang shivani.narang@ucalgary.ca
	Shivani Thakkar shivani.thakkar2@ucalgary.ca
QUERY: isha  
	Kishana Armstrong kishana.armstrong@ucalgary.ca
	Vishal Bhella vsbhella@ucalgary.ca
	Elisha Coonan elisha.coonan@ucalgary.ca
	Ian Hargreaves ishargre@ucalgary.ca
	Elisha Hawrys ehawrys@ucalgary.ca
	Seyede Javaheri Shalmani seyede.javaherishalm@ucalgary.ca
	Nisha John nisha.john@ucalgary.ca
	Alisha Jukes alisha.jukes@ucalgary.ca
	Anisha Karia-Venugopal anisha.kariavenugopa@ucalgary.ca
	Aisha Khan aisha.khan06@gmail.com
	Trisha Kloschinsky trisha.kloschinsky@ucalgary.ca
	Trisha Lichtenberger t.lichtenberger@ucalgary.ca
	Trisha McPherson tjmcpher@ucalgary.ca
	Alishaa Merchant alishaa.merchant@ucalgary.ca
	Isha Ober isha.ober@ucalgary.ca
	Amisha Patel amisha.patel2@ucalgary.ca
	Alisha Poole alisha.poole@ucalgary.ca
	Nishan Sharma nishan.sharma@ucalgary.ca
	Amuchou Soraisham amuchou.soraisham@calgaryhealthregion.ca
	Alisha Tessier No Email
	Vishal Tulsi vctulsi@ucalgary.c

,First Name,Last Name,Email,race,"Asian,GreaterEastAsian,EastAsian","Asian,GreaterEastAsian,Japanese","Asian,IndianSubContinent","GreaterAfrican,Africans","GreaterAfrican,Muslim","GreaterEuropean,British","GreaterEuropean,EastEuropean","GreaterEuropean,Jewish","GreaterEuropean,WestEuropean,French","GreaterEuropean,WestEuropean,Germanic","GreaterEuropean,WestEuropean,Hispanic","GreaterEuropean,WestEuropean,Italian","GreaterEuropean,WestEuropean,Nordic"
0,Shivani,Narang,shivani.narang@ucalgary.ca,"Asian,IndianSubContinent",0.000217,0.000045,0.986425,0.000539,0.008964,0.001219,0.000050,0.001326,0.000424,0.000063,0.000600,0.000065,0.000063
1,Shivani,Thakkar,shivani.thakkar2@ucalgary.ca,"Asian,IndianSubContinent",0.000094,0.000015,0.987320,0.000261,0.008818,0.001614,0.000032,0.000630,0.000324,0.000103,0.000694,0.000044,0.000049
2,Kishana,Armstrong,kishana.armstrong@ucalgary.ca,"GreaterEuropean,British",0.026097,0.025793,0.070332,0.016176,0.018025,0.712085,0.008959,0.029688,0.023698,0.008534,0.031393,0.007189,0.022030
3,Vishal,Bhella,vsbhella@ucalgary.ca,"Asian,IndianSubContinent",0.000436,0.000023,0.970158,0.001419,0.022171,0.003063,0.000420,0.000584,0.000199,0.000111,0.001082,0.000235,0.000099
4,Elisha,Coonan,elisha.coonan@ucalgary.ca,"GreaterEuropean,Jewish",0.002985,0.006562,0.011577,0.006651,0.020518,0.335704,0.007015,0.547231,0.022022,0.001290,0.024036,0.013284,0.001127
5,Ian,Hargreaves,ishargre@ucalgary.ca,"GreaterEuropean,British",0.002113,0.001562,0.001788,0.000910,0.001170,0.958243,0.000525,0.008837,0.003754,0.002318,0.014183,0.002563,0.002034
6,Elisha,Hawrys,ehawrys@ucalgary.ca,"GreaterEuropean,British",0.000512,0.003902,0.002721,0.004109,0.004706,0.742107,0.003682,0.189633,0.012967,0.001871,0.015482,0.017181,0.001127
7,Seyede,Javaheri Shalmani,seyede.javaherishalm@ucalgary.ca,"GreaterAfrican,Africans",0.001133,0.000787,0.112629,0.392263,0.133208,0.023878,0.001679,0.244825,0.018370,0.003116,0.014975,0.051851,0.001288
8,Nisha,John,nisha.john@ucalgary.ca,"Asian,IndianSubContinent",0.003267,0.002489,0.867104,0.003389,0.031053,0.051048,0.003199,0.025111,0.002348,0.002135,0.006219,0.001483,0.001156
9,Alisha,Jukes,alisha.jukes@ucalgary.ca,"GreaterEuropean,British",0.002041,0.004270,0.016613,0.026405,0.049065,0.666272,0.007777,0.168360,0.020263,0.003339,0.026902,0.006500,0.002192
